## Import

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
from scipy.stats import crystalball

## Define Useful functions

In [ ]:
def composite_gaussian(x, a1, mean1, sigma1, a2, mean2, sigma2):
    gauss1 = a1 * np.exp(-(x - mean1) ** 2 / (2 * sigma1 ** 2))
    gauss2 = a2 * np.exp(-(x - mean2) ** 2 / (2 * sigma2 ** 2))
    return gauss1 + gauss2

def gaussian_func(x, a, mu, sigma):
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))

def cball_func(x, a, mu, sigma, alpha, n):
    return a * crystalball.pdf(x, n, alpha, mu, sigma)

def return_closest_xvalue(x, value):
    return np.argmin(np.abs(x - value))

def single_gaussian_fit(x, y, fit_width):
    dy_dx = -1*np.gradient(y, x)
    max_index = np.argmax(dy_dx)
    selection = (dy_dx>0) & (x>=x[max_index]-fit_width) & (x<=x[max_index]+fit_width)

    initial_guess = [1.0, x[max_index], 1.0]
    popt, _ = curve_fit(gaussian_func, x[selection], dy_dx[selection], p0=initial_guess)

    return popt

## Load history

In [ ]:
con = sqlite3.connect('../ETROC-Data/ACCScurveHistory.sqlite')
df = pd.read_sql_query("SELECT * from acc_scurve", con)

## Select the board and print relevant notes

In [ ]:
chip_name = 'ET2_W36_IP5-17'
column_values = df[(df['chip_name'] == chip_name)]['note']

# Print the unique elements in the column
unique_elements = column_values.unique()
for element in unique_elements:
    print(element)

## Make plots and draw vertical lines from gaussian fit

In [ ]:
fit_width = 15

for element in unique_elements:
    selected_df = df[(df['chip_name'] == chip_name) & (df['note'] == element)]
    coupled_array = [(c, r) for c, r in zip(selected_df['col'], selected_df['row'])]
    
    fig = plt.figure(dpi=200, figsize=(20,10))
    u_cl = np.sort(np.unique(selected_df['col']))
    u_rl = np.sort(np.unique(selected_df['row']))
    gs = fig.add_gridspec(len(u_rl),len(u_cl))

    for ri, row_index in enumerate(u_rl):
        for ci, col_index in enumerate(u_cl):
            selected_row = selected_df[selected_df['row'] == row_index]
            selected_data = selected_row[selected_row['col'] == col_index]

            accvals = np.fromstring(selected_data['accscurve'].iloc[0], sep=',')
            dac_range = np.arange(selected_data['xmin'].iloc[0], selected_data['xmax'].iloc[0], selected_data['xstep'].iloc[0])
            time = pd.to_datetime(selected_data['timestamp'].iloc[0])
            time = time.strftime('%Y-%m-%d %H:%M:%S')

            popt = single_gaussian_fit(dac_range, accvals, fit_width)
            sigma = abs(popt[2])

            onesigmacdf = accvals[return_closest_xvalue(dac_range, np.ceil(popt[1] - sigma))] - accvals[return_closest_xvalue(dac_range, np.ceil(popt[1] + sigma))]
            twosigmacdf = accvals[return_closest_xvalue(dac_range, np.ceil(popt[1] - 2*sigma))] - accvals[return_closest_xvalue(dac_range, np.ceil(popt[1] + 2*sigma))]

            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1, len(u_cl)-ci-1])
            ax0.plot(dac_range, accvals, 'b.-', label="S-curve")
            ax0.axvline(popt[1], color='r', label=f"G. Peak Mean = {popt[1]:.1f}")
            ax0.axvline(popt[1]+sigma, color='k', label=fr"1$\sigma$ FW = {2*sigma:.1f}, {100*onesigmacdf/np.amax(accvals):.1f}%")
            ax0.axvline(popt[1]-sigma, color='k')
            ax0.axvline(popt[1]+2*sigma, color='k', ls="--", label=fr"2$\sigma$ FW = {4*sigma:.1f}, {100*twosigmacdf/np.amax(accvals):.1f}%")
            ax0.axvline(popt[1]-2*sigma, color='k', ls="--")
            ax0.legend()

            ax0.text(0.9, 0.5, time, transform=ax0.transAxes, ha='right', va='center', fontsize=8)
            ax0.text(0.9, 0.465, f'Row:{row_index}, Col:{col_index}', transform=ax0.transAxes, ha='right', va='center', fontsize=8)

    fig.text(0.5, 0.001, "DAC Value [LSB]", ha='center', va='center', fontsize=12)
    fig.text(0.001, 0.5, "ACC Value [decimal]", ha='center', va='center', rotation='vertical', fontsize=12)

    plt.suptitle(element, fontsize=12)
    plt.tight_layout()

plt.show()